<img src= "https://i.ytimg.com/vi/f80oFppiWS0/maxresdefault.jpg">Проект по DS(SR)  .Места в городе Манчестер в которых были преступления</img>

<h3> by Momunkulov Z. CS1801K</h3>

<h4> Вступление </h4>

Отвечу на несколько вопросов, город Манчестер я выбрал по принципу футбольного клуба. Именно там был основан клуб Manchester United. 

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/7/7a/Manchester_United_FC_crest.svg/220px-Manchester_United_FC_crest.svg.png"></img>


<h2> Подгружаем библиотеки </h2>

In [1]:
import numpy as np  
import pandas as pd 

<h2> Мы будем работать с Folium , а значит надо и установить и его</h2>

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: ...working... done

# All requested packages already installed.

Folium installed and imported!


<h2> Проверяем всё ли работает, прописывая значения </h2>

In [3]:
mu_map = folium.Map()
mu_map

<h2> По данным из сайта <a href="https://www.latlong.net/place/manchester-uk-4194.html">Latlong</a> мы находим координаты города Манчестер: 53.483959 и -2.244644 . И установим значение zoom_start - 12.

In [4]:
mu_map = folium.Map(location=[53.483959, -2.244644], zoom_start=12)
mu_map

<h2> Можно по эксперементировать </h2> 

Можно по экспериментировать с картами, так к примеру есть виды карт "Stamen"а. По какой-то причине они не работают с казахстанских IP , поэтому чтобы их подгрузить я использовал VPN. В Stamen есть 6 вида но я реализовал следующее:
    
    Toner(1-ое фото)
    Watercolor(2-ое фото)
    Terrain(3-ье фото)

In [5]:
mu_map = folium.Map(location=[53.483959, -2.244644], zoom_start=12, tiles = 'Stamen Toner')
mu_map

In [6]:
mu_map = folium.Map(location=[53.483959, -2.244644], zoom_start=12, tiles = 'Stamen Watercolor')
mu_map

In [7]:
mu_map = folium.Map(location=[53.483959, -2.244644], zoom_start=12, tiles = 'Stamen Terrain')
mu_map

In [8]:
mu_map = folium.Map(location=[53.483959, -2.244644], zoom_start=12)

<h2> Использованная дата </h2>

На моё удивление, я нашёл национальную библеотеку данных, однако они не смогли мне предоставить данные именно в csv формате(или в любом другом которую поддерживает python). Поэтому поискав я нашёл базу <a href= "https://data.police.uk/data/">полицейских записей по регионам и координатам</a>, я решил выбрать именно её. 

In [9]:
df_data = pd.read_csv('2018-01-greater-manchester-street.csv')
print('Data imported successfully')

Data imported successfully


<h3>Дата была взята со всего Большого Манчестера, но нас интересует в основном Longitude и Latitude</h3>


In [10]:
df_data.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crimetype,Last outcome category,Context;
0,9c48e3687788c1930bca1f1b6efdaea2882eeb89f44b42...,2018-01,Greater Manchester Police,Greater Manchester Police,-2.462774,53.622101,On or near Scout Road,E01012628,Blackburn with Darwen 018D,Other theft,Investigation complete; no suspect identified,NaN
1,f0df3bd4a2d5123c5cebcb0f22361812b9d9117dcab47e...,2018-01,Greater Manchester Police,Greater Manchester Police,-2.462774,53.622101,On or near Scout Road,E01012628,Blackburn with Darwen 018D,Robbery,Court result unavailable,;
2,0741743b9ee81d4ef8ffc170a090bd646aa1e4afa5797f...,2018-01,Greater Manchester Police,Greater Manchester Police,-2.462774,53.622101,On or near Scout Road,E01012628,Blackburn with Darwen 018D,Violence and sexual offences,Offender given community sentence,;
3,3a6d3c03d45992e0e8592af145ec284b863e15112295f0...,2018-01,Greater Manchester Police,Greater Manchester Police,-2.444683,53.611159,On or near Horrocks Fold Avenue,E01004768,Bolton 001A,Burglary,Investigation complete; no suspect identified,NaN
4,69943711a03fa83acbb18325338e7845543a90c0df7526...,2018-01,Greater Manchester Police,Greater Manchester Police,-2.440493,53.612388,On or near Threlkeld Road,E01004768,Bolton 001A,Burglary,Investigation complete; no suspect identified,NaN


<h3>Командой .shape проверяет кол-во всех строк </h3>

In [11]:
df_data.shape

(31774, 12)

<h2> Переизбыток данных </h2>

Так как я выбрал город Манчестер(а именно Большой Манчестер) в обязанности хранителей порядка входит абсолютно ВСЕ районы Б.М. Это и Виган, Бирмингам и собственно сам Манчестер. Так как обрабатывать все 31774 данных будет сложно(и почти невозможно) я решил найти в базе LSOA (Lower Layer Super Output) или другими словами это код которым обозначают регионы стражи правопорядка. Теперь мы делаем поиск и получаем значения только в регионе Манчестера.
<a href= "http://geoportal.statistics.gov.uk/datasets/cc93d1e3d0e249288e6a845b87d7252d_0/data?orderBy=WD16NM&page=3&where=WD16NM%20like%20%27%25Hulme%25%27">UK GeoPortal Statistics portal</a> покажет все такие LSOA коды. Я для своего примера взял "Hulme"(Manchester 019D).


In [12]:
df_data = pd.read_csv("2018-01-greater-manchester-street.csv", sep = ",")
df_data = df_data[df_data["LSOA name"] == 'Manchester 019D' ] 
#print(df_data[df_data["Latitude"] == 53.622101])


<h3> И снова проверим </h3>

In [13]:
df_data.shape

(25, 12)

<h2> Теперь с помощью следующих команд мы отрисуем точки на карте </h2>

In [14]:
policerep = folium.map.FeatureGroup()

for lat, lng, in zip(df_data.Latitude, df_data.Longitude):
    policerep.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='black',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

# add incidents to map
mu_map.add_child(policerep)

In [15]:

policerep = folium.map.FeatureGroup()

for lat, lng, in zip(df_data.Latitude, df_data.Longitude):
    policerep.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='black',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

latitudes = list(df_data.Latitude)
longitudes = list(df_data.Longitude)
labels = list("Crime: " + df_data.Crimetype + ". Location: " + df_data.Location)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(mu_map)    
    
mu_map.add_child(policerep)

<h2> Здесь мы указываем регионы(выделяем территории) </h2>

In [19]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
mu_map = folium.Map(location = [latitudes, longitudes], zoom_start = 4)

# instantiate a mark cluster object for the incidents in the dataframe
policerep = plugins.MarkerCluster().add_to(mu_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(latitudes, longitudes, labels):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(policerep)

# display map
mu_map

<h2> Итог </h2>

На этом я считаю что проект завершен. На карте всё показывается и отрисовывает. 

<h2>Использованные ссылки:</h2>

   <a href= "https://data.police.uk/data/">UK Police Data</a> - Сайт с записями полицейских департаментов во всей Великобритании.
   
   <a href="https://www.latlong.net/place/manchester-uk-4194.html">Latlong</a> - Сайт с координатами(долгота и ширина) по запросу
    